# Feed the results of the ESM back in the LCI database 
The construction and operation of technologies at year t as well as the utilization of resources at year t are allocated to year t. Thus, those shall not be included again in order to avoid double-counting. However, new constructions at year t+1 are using the mix build over the past iterations. This mix is defined by the shares resulting from the ESM at the previous iteration. As mentioned before, the construction phase should be set to zero as it was allocated to the construction year.  

In [1]:
import pandas as pd
import bw2data as bd
from mescal import *

In [2]:
bd.projects.set_current("ei3.8-mescal")

In [3]:
annual_prod = pd.read_csv('energyscope_data/results_ES.csv')
model = pd.read_csv('energyscope_data/model.csv')
mapping = pd.read_csv('energyscope_data/mapping_linked.csv')
flows = pd.read_csv('energyscope_data/hidden/flows_ecoinvent.csv')

In [4]:
name_premise_with_CPC_db = 'ecoinvent_cutoff_3.8_remind_SSP2-Base_2020_with_CPC'

In [5]:
premise_db_with_CPC = load_extract_db(name_premise_with_CPC_db)

In [6]:
db_mapping = concatenate_databases(list(mapping.Database.unique()))
db_flows = concatenate_databases(list(flows.Database.unique()))

In [7]:
flows['Type'] = len(flows) * ['Flow']

In [8]:
# Define the user-defined ranking
my_ranking = [
    'CA-QC', # Quebec
    'CA', # Canada
    'CA-ON', # Other canadian provinces 
    'CA-AB',
    'CA-BC',
    'CA-MB',
    'CA-NB',
    'CA-NF',
    'CA-NS',
    'CA-NT',
    'CA-NU',
    'CA-PE',
    'CAZ', # Canada - Australia - New Zealand
    'RNA', # North America
    'US', # United States
    'USA', # United States
    'GLO', # Global average 
    'RoW', # Rest of the world
]

In [9]:
flows = change_location_mapping_file(
    flows,
    my_ranking,
    db_flows,
    'QC',
)

In [10]:
flows = create_complementary_database(flows, premise_db_with_CPC, '')

The complementary database did not needed to be created


In [11]:
flows.head()

,Name,Type,Product,Activity,Location,Database
0,ACETIC_ACID,Flow,"acetic acid, without water, in 98% solution state","market for acetic acid, without water, in 98% ...",GLO,ecoinvent_cutoff_3.8_remind_SSP2-Base_2020_wit...
1,ACETONE,Flow,"acetone, liquid","market for acetone, liquid",RoW,ecoinvent_cutoff_3.8_remind_SSP2-Base_2020_wit...
2,ALUMINUM,Flow,"aluminium, primary, ingot","market for aluminium, primary, ingot",RoW,ecoinvent_cutoff_3.8_remind_SSP2-Base_2020_wit...
3,BENZENE,Flow,benzene,market for benzene,GLO,ecoinvent_cutoff_3.8_remind_SSP2-Base_2020_wit...
4,BIO_DIESEL,Flow,"biodiesel, vehicle grade","biodiesel, from used cooking oil, at fuelling ...",RER,ecoinvent_cutoff_3.8_remind_SSP2-Base_2020_wit...


In [12]:
for i in range(len(flows)):
    db = create_or_modify_activity_from_esm_results(
        db=db_mapping,
        original_activity_prod=flows.Product.iloc[i],
        original_activity_name=flows.Activity.iloc[i],
        original_activity_database=flows.Database.iloc[i],
        model=model,
        flows=flows,
        esm_results=annual_prod,
        mapping=mapping,
    )

ACETIC_ACID
ETHANE_OXIDATION
METHANOL_CARBONYLATION
ACETONE
CUMENE_PROCESS
AL_MAKING
AL_MAKING_HR
BENZENE
METHANOL_TO_AROMATICS
BIO_DIESEL
CEMENT_PROD
CEMENT_PROD_HP
DAC_HT
DAC_LT
STO_CO2
COAL
DIESEL
CO2_TO_DIESEL
FT
STO_DIE
ELECTRICITY_EHV
CCGT
CCGT_CC
COAL_IGCC
COAL_IGCC_CC
COAL_US
COAL_US_CC
H2_CCGT
H2_NG_CCGT
H2_NG_CCGT_CCS
HYDRO_DAM
HYDRO_RIVER
NEW_HYDRO_DAM
OCGT_LARGE
OCGT_SMALL
TRAFO_MH
NUCLEAR
ELECTRICITY_EHV
CCGT
CCGT_CC
COAL_IGCC
COAL_IGCC_CC
COAL_US
COAL_US_CC
H2_CCGT
H2_NG_CCGT
H2_NG_CCGT_CCS
HYDRO_DAM
HYDRO_RIVER
NEW_HYDRO_DAM
OCGT_LARGE
OCGT_SMALL
TRAFO_MH
NUCLEAR
PV
TRAFO_ML
STO_ELEC
AN_DIG_SI
GEOTHERMAL
HYDRO_GAS_CHP
NEW_HYDRO_RIVER
TIDAL
TRAFO_HM
TRAFO_LM
WIND_OFFSHORE
WIND_ONSHORE
ETHANE
ETHANOL
CROPS_TO_ETHANOL
ETHYLBENZENE
ETHYLENE
ETHANE_CRACKING
ETHANE_TO_ETHYLENE
METATHESIS_PROPYLENE
METHANOL_TO_OLEFINS
FOOD_PROD
FOOD_PROD_HP
FOOD_PROD_HR
GASOLINE
STO_GASO
H2_EHP
H2_HP
H2_MP
H2_LP
ALKALINE_ELECTROLYSIS
ATR
ATR_CCS
BIOGAS_ATR
BIOGAS_ATR_CCS
BIOGAS_SMR
BIOGAS_SMR_C

IndexError: single positional indexer is out-of-bounds